## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-15-01-15-22 +0000,wapo,Iranians abroad wait for news from loved ones ...,https://www.washingtonpost.com/world/2026/01/1...
1,2026-01-15-01-13-50 +0000,nypost,Hochul wants to open medical marijuana center ...,https://nypost.com/2026/01/14/us-news/hochul-w...
2,2026-01-15-01-12-38 +0000,wapo,Protesters condemn ICE tactics as Minneapolis ...,https://www.washingtonpost.com/nation/2026/01/...
3,2026-01-15-01-12-02 +0000,nypost,NASA sends 4 astronauts back to Earth in first...,https://nypost.com/2026/01/14/us-news/nasa-con...
4,2026-01-15-01-11-04 +0000,nyt,"What Voters Told Democrats About ICE, Rising C...",https://www.nytimes.com/2026/01/14/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2404/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
53,trump,68
175,iran,38
139,new,17
171,greenland,16
54,venezuela,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
71,2026-01-14-22-56-00 +0000,wsj,Top officials from Denmark and Greenland said ...,https://www.wsj.com/politics/national-security...,143
19,2026-01-15-00-54-00 +0000,wsj,The Senate voted to kill a war-powers resoluti...,https://www.wsj.com/politics/policy/trump-beat...,139
46,2026-01-14-23-50-00 +0000,wsj,Oil Slips After Trump Appears to Tilt Away Fro...,https://www.wsj.com/finance/commodities-future...,132
68,2026-01-14-23-04-40 +0000,nyt,U.S. Moves Some Personnel From Qatar Air Base ...,https://www.nytimes.com/2026/01/14/us/politics...,132
151,2026-01-14-20-17-00 +0000,wsj,"Fed governor Stephen Miran, an appointee of Pr...",https://www.wsj.com/economy/central-banking/fe...,131


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
71,143,2026-01-14-22-56-00 +0000,wsj,Top officials from Denmark and Greenland said ...,https://www.wsj.com/politics/national-security...
286,75,2026-01-14-13-02-49 +0000,nypost,Doctors in Iran collapsing from exhaustion as ...,https://nypost.com/2026/01/14/world-news/iran-...
19,62,2026-01-15-00-54-00 +0000,wsj,The Senate voted to kill a war-powers resoluti...,https://www.wsj.com/politics/policy/trump-beat...
137,57,2026-01-14-20-50-09 +0000,bbc,US launches phase two of Gaza peace plan with ...,https://www.bbc.com/news/articles/cjw1nxe5pvlo...
151,50,2026-01-14-20-17-00 +0000,wsj,"Fed governor Stephen Miran, an appointee of Pr...",https://www.wsj.com/economy/central-banking/fe...
115,49,2026-01-14-21-44-49 +0000,cbc,Renee Nicole Good's family hires George Floyd ...,https://www.cbc.ca/news/world/renee-nicole-goo...
228,45,2026-01-14-16-36-48 +0000,nypost,Home sales slump dragged through 2025 as mortg...,https://nypost.com/2026/01/14/business/home-sa...
247,41,2026-01-14-15-23-05 +0000,nyt,China Announces Record Trade Surplus as Its Ex...,https://www.nytimes.com/2026/01/13/business/ch...
188,41,2026-01-14-18-27-54 +0000,latimes,Child porn concerns prompt California to launc...,https://www.latimes.com/california/story/2026-...
318,38,2026-01-14-11-00-56 +0000,wapo,You don’t need Trump to cap your credit card i...,https://www.washingtonpost.com/business/2026/0...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
